# Data pre-processing pipeline

In this notebook, we are going to:
- give the following inputs:
    - `RAW.movie_data_scraped`, `RAW.character_classification` and `RAW.summaries`
- receive the following output:
    - `clean_data`

`RAW.movie_data_scraped` is the product of the movie metadata upgraded through scraping (see `scraping.ipynb`).


`RAW.character_classification` is the product of the parsing of `RAW.summaries` which leads to character classification (see `parsing_flair.ipynb`)

`RAW.summaries` is used for the very first step of the data pre-processing (see point 1.)

`clean_data` is the data points for our models (see `model_training.ipynb`)

### Library imports

In [199]:
import pandas as pd
import numpy as np
import cpi
import re
from SPARQLWrapper import SPARQLWrapper, JSON
import itertools

import dataframes as RAW

### Inputs

In [200]:
movies = RAW.movie_data_scraped.copy()
characters= pd.read_csv('../Clean data/character_classification.csv')
summaries = RAW.summaries.copy()

In [201]:
characters.shape

(34000, 14)

### Data pre-processing

1. Dataframe `movies`

1.1. Keep movies which have a summary (not needed for characters as this is done in `parsing_flair.ipynb`). Gets rid of 16.429 movies.

In [202]:
movies.shape

(39372, 12)

In [203]:
movies = movies[movies['Wiki ID'].isin(summaries['Wiki ID'])]

In [204]:
movies.shape

(22943, 12)

1.2 Revenue

Adapt to inflation:

- CPI can only take into account data after 1913. Hence all movies before are filtered out. Gets rid of 53 movies

In [205]:
movies.shape

(22943, 12)

In [206]:
def get_revenues_adjusted(data):
    # Create a new column 'Release year' and initialize it to None
    data['Release year'] = None
    
    # Iterate through each row in the dataset
    for index, row in data.iterrows():
        release_date = row['Release date']
        
        # Check the type of value in the 'release date' column
        if pd.notna(release_date) and isinstance(release_date, float):
            # If it's a non-NaN float, directly assign this value to the new 'Release year' column
            data.at[index, 'Release year'] = int(release_date)
        elif pd.notna(release_date) and isinstance(release_date, str):
            # If it's a non-NaN string, convert it to datetime and extract the year
            release_date = pd.to_datetime(release_date)
            data.at[index, 'Release year'] = release_date.year

    # Filter the dataframe for entries after 1913 to ensure consistent revenue data
    data = data[data['Release year'] >= 1913]
    
    # Add a new column 'Revenue' adjusted with the inflation rate
    data['Revenue'] = data.apply(lambda x: cpi.inflate(x['Revenue'], x['Release year']), axis=1)
    return data

movies = get_revenues_adjusted(movies)


<ipython-input-206-77cb126e0d86>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Revenue'] = data.apply(lambda x: cpi.inflate(x['Revenue'], x['Release year']), axis=1)


In [207]:
movies.shape

(22890, 13)

1.3 Runtime

Keep movies within an acceptable range. Gets rid of 524 movies

In [208]:
movies.shape

(22890, 13)

In [209]:
# Create a function as it will be used multiple times
def Outliers_filtering(data, subject, min_val, max_val):
    # Filter the data based on conditions:
    # If the 'subject' column is null or falls within the range [min_val, max_val]
    filtered_data = data[(data[subject].isnull()) | ((data[subject] >= min_val) & (data[subject] <= max_val))]
    return filtered_data


In [210]:
movies = Outliers_filtering(movies, 'Runtime', 15, 300)

In [211]:
movies.shape

(22366, 13)

1.4 Languages

Change format (also done at the same time for Genres and Countries)

In [212]:
movies.shape

(22366, 13)

In [213]:
def dict_to_list(data, column):
    # Filter the data where the specified column is not null
    filtered_data = data[data[column].notnull()]
    
    # Apply regex to extract specific patterns from the column values
    filtered_data[column] = filtered_data[column].apply(lambda x: re.findall(r'": "(.*?)"', x))
    
    # Select only the 'Wiki ID' and the specified column
    filtered_data = filtered_data[['Wiki ID', column]]
    
    return filtered_data


In [214]:
# Merge 'movies' dataframe with specific columns converted into lists using the 'dict_to_list' function

# Merge 'Languages' column into 'movies' dataframe based on 'Wiki ID'
movies = pd.merge(movies, dict_to_list(movies, 'Languages'), on='Wiki ID', how='left')

# Merge 'Genres' column into 'movies' dataframe based on 'Wiki ID'
movies = pd.merge(movies, dict_to_list(movies, 'Genres'), on='Wiki ID', how='left')

# Merge 'Countries' column into 'movies' dataframe based on 'Wiki ID'
movies = pd.merge(movies, dict_to_list(movies, 'Countries'), on='Wiki ID', how='left')

# Drop columns from 'movies' dataframe by name
columns_to_drop = ['Languages_x', 'Countries_x', 'Genres_x']  # List of columns to be dropped
movies.drop(columns=columns_to_drop, inplace=True)  # Drop the specified columns

# Rename columns in 'movies' dataframe
column_dict = {
    'Languages_y': 'Languages',  # Rename 'Languages_y' to 'Languages'
    'Genres_y': 'Genres',        # Rename 'Genres_y' to 'Genres'
    'Countries_y': 'Countries'   # Rename 'Countries_y' to 'Countries'
}
movies.rename(columns=column_dict, inplace=True)  # Apply column renaming


In [215]:
movies.shape

(22366, 13)

Get rid of silent film as a language

In [216]:
# Filter rows where 'Language' contains [Silent film] 
movies['Languages'] = movies['Languages'].apply(lambda x: [lang for lang in x if lang != 'Silent film'])

Keep only certain languages above a selected threshold. Gets rid of 2355 movies.

In [217]:
# Counts the numbers of appearance of each genre in all movies
exploded_languages = movies['Languages'].explode()
languages_counts = exploded_languages.value_counts()

# A quick look at the numbers of appearance of each genre in all movies
languages_counts

Languages
English Language      14955
Hindi Language         1195
French Language        1146
Spanish Language        868
Japanese Language       804
                      ...  
Krio Language             1
Australian English        1
Yolngu Matha              1
Shanxi                    1
Haryanvi Language         1
Name: count, Length: 184, dtype: int64

In [218]:
movies.shape

(22366, 13)

In [219]:
# List of languages before threshold filtering
unique_languages_list = languages_counts.index.tolist()

unique_languages_list

['English Language',
 'Hindi Language',
 'French Language',
 'Spanish Language',
 'Japanese Language',
 'German Language',
 'Italian Language',
 'Korean Language',
 'Tamil Language',
 'Telugu language',
 'Standard Mandarin',
 'Russian Language',
 'Malayalam Language',
 'Cantonese',
 'Urdu Language',
 'Arabic Language',
 'Swedish Language',
 'Bengali Language',
 'Standard Cantonese',
 'Portuguese Language',
 'Tagalog language',
 'Mandarin Chinese',
 'Thai Language',
 'American English',
 'Hebrew Language',
 'Danish Language',
 'Latin Language',
 'Dutch Language',
 'Punjabi language',
 'Turkish Language',
 'Filipino language',
 'Polish Language',
 'Norwegian Language',
 'Greek Language',
 'Czech Language',
 'Chinese language',
 'Serbian language',
 'Indonesian Language',
 'Finnish Language',
 'Kannada Language',
 'Persian Language',
 'Malay Language',
 'Hungarian language',
 'Vietnamese Language',
 'Icelandic Language',
 'Romanian Language',
 'Marathi Language',
 'American Sign Language'

In [220]:
movies.shape

(22366, 13)

In [221]:
# Threshold of 100 mentions
threshold = 100
# Filter out languages below the threshold
filtered_languages = languages_counts[languages_counts >= threshold].index.tolist()

# Creates filter for the original DataFrame based on the threshold
filtered_languages = movies.apply(lambda row: [language for language in row['Languages'] if language in filtered_languages], axis=1)

In [222]:
# Keep only above threshold languages
movies["Languages"] = filtered_languages
movies

,Wiki ID,Freebase ID,Movie name,Release date,Revenue,Runtime,IMDb rating,Wins,Nominations,Release year,Languages,Genres,Countries
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,2.315268e+07,98.0,4.9,NaN,2.0,2001,[English Language],"[Thriller, Science Fiction, Horror, Adventure,...",[United States of America]
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,5.8,NaN,NaN,1997,[English Language],"[Romantic comedy, Ensemble Film, Comedy-drama,...",[United States of America]
2,171005,/m/016ywb,Henry V,1989-11-08,2.398142e+07,137.0,7.5,12.0,13.0,1989,[English Language],"[Costume drama, War film, Epic, Period piece, ...",[United Kingdom]
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,9.655040e+08,139.0,7.8,22.0,17.0,1964,[English Language],"[Children's/Family, Musical, Fantasy, Comedy, ...",[United States of America]
4,20604092,/m/0523t_1,Anbu Thozhi,2007-08-17,NaN,NaN,NaN,NaN,NaN,2007,[Tamil Language],[Romance Film],[India]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22361,26482675,/m/0bbwngb,Eşrefpaşalılar,2010-03-05,2.479777e+06,NaN,4.9,NaN,NaN,2010,[],"[Comedy film, Drama]",[]
22362,23851782,/m/06_vb43,The Ghost Train,1941-05-03,NaN,82.0,6.2,NaN,NaN,1941,[English Language],"[Crime Fiction, Thriller, Comedy, Supernatural]",[United Kingdom]
22363,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,4.6,NaN,NaN,2011,[English Language],[Drama],[United States of America]
22364,34980460,/m/0g4pl34,Knuckle,2011-01-21,3.443857e+03,96.0,6.8,1.0,4.0,2011,[English Language],"[Biographical film, Drama, Documentary]","[Ireland, United Kingdom]"


In [223]:
movies.shape

(22366, 13)

In [224]:
# Counts the counts for each language in all movies after threshold filter
exploded_languages = movies['Languages'].explode()
language_counts = exploded_languages.value_counts()
language_counts

Languages
English Language       14955
Hindi Language          1195
French Language         1146
Spanish Language         868
Japanese Language        804
German Language          610
Italian Language         542
Korean Language          526
Tamil Language           500
Telugu language          404
Standard Mandarin        337
Russian Language         308
Malayalam Language       281
Cantonese                251
Urdu Language            163
Arabic Language          157
Swedish Language         143
Bengali Language         141
Standard Cantonese       128
Portuguese Language      122
Tagalog language         102
Name: count, dtype: int64

In [225]:
movies.shape

(22366, 13)

In [226]:
# Filter 'movies' DataFrame to retain rows where the 'Languages' column has non-empty values
movies = movies[movies['Languages'].astype(bool)]

# Display the first 30 rows of the filtered DataFrame
movies.head(30)

,Wiki ID,Freebase ID,Movie name,Release date,Revenue,Runtime,IMDb rating,Wins,Nominations,Release year,Languages,Genres,Countries
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,2.315268e+07,98.0,4.9,NaN,2.0,2001,[English Language],"[Thriller, Science Fiction, Horror, Adventure,...",[United States of America]
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,5.8,NaN,NaN,1997,[English Language],"[Romantic comedy, Ensemble Film, Comedy-drama,...",[United States of America]
2,171005,/m/016ywb,Henry V,1989-11-08,2.398142e+07,137.0,7.5,12.0,13.0,1989,[English Language],"[Costume drama, War film, Epic, Period piece, ...",[United Kingdom]
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,9.655040e+08,139.0,7.8,22.0,17.0,1964,[English Language],"[Children's/Family, Musical, Fantasy, Comedy, ...",[United States of America]
4,20604092,/m/0523t_1,Anbu Thozhi,2007-08-17,NaN,NaN,NaN,NaN,NaN,2007,[Tamil Language],[Romance Film],[India]
5,156558,/m/014k4y,Baby Boy,2001-06-27,4.855272e+07,123.0,6.4,1.0,14.0,2001,[English Language],"[Crime Fiction, Drama, Coming of age]",[United States of America]
6,11448183,/m/02rc_h4,Bindiya Chamkegi,1984-01-20,NaN,NaN,4.8,NaN,NaN,1984,[Hindi Language],"[Family Film, Comedy]",[India]
7,29772142,/m/0fq260_,Karayilekku Oru Kadal Dooram,2010-12-31,NaN,110.0,6.1,NaN,NaN,2010,[Malayalam Language],[Drama],[India]
8,26623942,/m/0bh9fbk,Kausthubham,2010-03-19,NaN,NaN,NaN,NaN,NaN,2010,[Malayalam Language],[Drama],[India]
9,9548445,/m/02pjlrp,Archie: To Riverdale and Back Again,1990-05-06,NaN,100.0,5.3,NaN,NaN,1990,[English Language],[Comedy],[United States of America]


In [227]:
movies.shape

(20011, 13)

1.5 Countries

No need to reformat as it was done in the first steps of the Languages section.

Some countries have to be mapped as the country name is outdated (some mapping can be debated).

In [228]:
# Manually-categorised countries
countries_to_modify = ['Hong Kong', 'West Germany', 'Czechoslovakia', 'Soviet Union', 'Yugoslavia']

# Dictionary to map countries
correspondances_countries = {
    'West Germany': 'Germany',
    'Czechoslovakia': 'Czech Republic',
    'England': 'United Kingdom',
    'Soviet Union': 'Russia',
    'Yugoslavia': 'Serbia'
}

# Function to map countries in the list
def map_countries(country_list):
    return [correspondances_countries.get(country, country) for country in country_list]

# Apply the mapping function to the 'Countries' column
movies['Countries'] = movies['Countries'].apply(map_countries)

<ipython-input-228-d8bbdc020ee5>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['Countries'] = movies['Countries'].apply(map_countries)


Keep only certain countries above a selected threshold. Gets rid of 650 movies.

In [229]:
# Counts the numbers of appearance of each country in all movies
exploded_countries = movies['Countries'].explode()
countries_counts = exploded_countries.value_counts()

# A quick look at the numbers of appearance of each country in all movies
countries_counts

Countries
United States of America    12397
India                        2200
United Kingdom               1956
France                       1170
Germany                       887
                            ...  
Nigeria                         1
Guinea-Bissau                   1
Congo                           1
Jordan                          1
Iraq                            1
Name: count, Length: 120, dtype: int64

In [230]:
# List of countries before threshold filtering
unique_countries_list = countries_counts.index.tolist()

unique_countries_list

['United States of America',
 'India',
 'United Kingdom',
 'France',
 'Germany',
 'Japan',
 'Canada',
 'Italy',
 'South Korea',
 'Australia',
 'Spain',
 'Hong Kong',
 'China',
 'Sweden',
 'Argentina',
 'Mexico',
 'New Zealand',
 'Philippines',
 'Russia',
 'Ireland',
 'Belgium',
 'Denmark',
 'Netherlands',
 'United Kingodm',
 'Brazil',
 'South Africa',
 'Switzerland',
 'Austria',
 'Czech Republic',
 'Pakistan',
 'Taiwan',
 'Norway',
 'Israel',
 'Poland',
 'Luxembourg',
 'German Democratic Republic',
 'Singapore',
 'Hungary',
 'Finland',
 'Portugal',
 'Egypt',
 'Weimar Republic',
 'Serbia',
 'Romania',
 'Thailand',
 'Korea',
 'Morocco',
 'Malaysia',
 'Greece',
 'Scotland',
 'Turkey',
 'Iceland',
 'Colombia',
 'Algeria',
 'Bulgaria',
 'Venezuela',
 'United Arab Emirates',
 'Bangladesh',
 'Chile',
 'Puerto Rico',
 'Lebanon',
 'Slovenia',
 'Kingdom of Great Britain',
 'Peru',
 'Cuba',
 'Tunisia',
 'Croatia',
 'Slovakia',
 'Uruguay',
 'Ukraine',
 'Bosnia and Herzegovina',
 'Democratic Republ

In [231]:
movies.shape

(20011, 13)

In [232]:
# Threshold of 60
threshold = 60
# Filter out genres below the threshold
filtered_countries = countries_counts[countries_counts >= threshold].index.tolist()

# Creates filter for the original DataFrame based on the threshold
filtered_countries = movies.apply(lambda row: [country for country in row['Countries'] if country in filtered_countries], axis=1)

In [233]:
movies.shape

(20011, 13)

In [234]:
#Inject the modified "Countries"
movies["Countries"] = filtered_countries
movies

<ipython-input-234-b0e773d32d79>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["Countries"] = filtered_countries


,Wiki ID,Freebase ID,Movie name,Release date,Revenue,Runtime,IMDb rating,Wins,Nominations,Release year,Languages,Genres,Countries
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,2.315268e+07,98.0,4.9,NaN,2.0,2001,[English Language],"[Thriller, Science Fiction, Horror, Adventure,...",[United States of America]
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,5.8,NaN,NaN,1997,[English Language],"[Romantic comedy, Ensemble Film, Comedy-drama,...",[United States of America]
2,171005,/m/016ywb,Henry V,1989-11-08,2.398142e+07,137.0,7.5,12.0,13.0,1989,[English Language],"[Costume drama, War film, Epic, Period piece, ...",[United Kingdom]
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,9.655040e+08,139.0,7.8,22.0,17.0,1964,[English Language],"[Children's/Family, Musical, Fantasy, Comedy, ...",[United States of America]
4,20604092,/m/0523t_1,Anbu Thozhi,2007-08-17,NaN,NaN,NaN,NaN,NaN,2007,[Tamil Language],[Romance Film],[India]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22360,15394941,/m/03m6zh4,Gopi Kishan,1994-12-02,NaN,NaN,5.8,NaN,NaN,1994,[Hindi Language],"[Drama, Action, Crime, Comedy]",[India]
22362,23851782,/m/06_vb43,The Ghost Train,1941-05-03,NaN,82.0,6.2,NaN,NaN,1941,[English Language],"[Crime Fiction, Thriller, Comedy, Supernatural]",[United Kingdom]
22363,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,4.6,NaN,NaN,2011,[English Language],[Drama],[United States of America]
22364,34980460,/m/0g4pl34,Knuckle,2011-01-21,3.443857e+03,96.0,6.8,1.0,4.0,2011,[English Language],"[Biographical film, Drama, Documentary]","[Ireland, United Kingdom]"


In [235]:
# Counts the counts for each country in all movies after threshold filter
exploded_countries = movies['Countries'].explode()
country_counts = exploded_countries.value_counts()
country_counts

Countries
United States of America    12397
India                        2200
United Kingdom               1956
France                       1170
Germany                       887
Japan                         751
Canada                        744
Italy                         559
South Korea                   504
Australia                     332
Spain                         310
Hong Kong                     290
China                         154
Sweden                        147
Argentina                     130
Mexico                        112
New Zealand                   106
Philippines                    99
Russia                         98
Ireland                        95
Belgium                        80
Denmark                        69
Netherlands                    67
United Kingodm                 60
Brazil                         60
Name: count, dtype: int64

In [236]:
movies.shape

(20011, 13)

In [237]:
# Filter 'movies' DataFrame to retain rows where the 'Countries' column has non-empty values
movies = movies[movies['Countries'].astype(bool)]

# Display the first 30 rows of the filtered DataFrame
movies.head(30)

,Wiki ID,Freebase ID,Movie name,Release date,Revenue,Runtime,IMDb rating,Wins,Nominations,Release year,Languages,Genres,Countries
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,2.315268e+07,98.0,4.9,NaN,2.0,2001,[English Language],"[Thriller, Science Fiction, Horror, Adventure,...",[United States of America]
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,5.8,NaN,NaN,1997,[English Language],"[Romantic comedy, Ensemble Film, Comedy-drama,...",[United States of America]
2,171005,/m/016ywb,Henry V,1989-11-08,2.398142e+07,137.0,7.5,12.0,13.0,1989,[English Language],"[Costume drama, War film, Epic, Period piece, ...",[United Kingdom]
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,9.655040e+08,139.0,7.8,22.0,17.0,1964,[English Language],"[Children's/Family, Musical, Fantasy, Comedy, ...",[United States of America]
4,20604092,/m/0523t_1,Anbu Thozhi,2007-08-17,NaN,NaN,NaN,NaN,NaN,2007,[Tamil Language],[Romance Film],[India]
5,156558,/m/014k4y,Baby Boy,2001-06-27,4.855272e+07,123.0,6.4,1.0,14.0,2001,[English Language],"[Crime Fiction, Drama, Coming of age]",[United States of America]
6,11448183,/m/02rc_h4,Bindiya Chamkegi,1984-01-20,NaN,NaN,4.8,NaN,NaN,1984,[Hindi Language],"[Family Film, Comedy]",[India]
7,29772142,/m/0fq260_,Karayilekku Oru Kadal Dooram,2010-12-31,NaN,110.0,6.1,NaN,NaN,2010,[Malayalam Language],[Drama],[India]
8,26623942,/m/0bh9fbk,Kausthubham,2010-03-19,NaN,NaN,NaN,NaN,NaN,2010,[Malayalam Language],[Drama],[India]
9,9548445,/m/02pjlrp,Archie: To Riverdale and Back Again,1990-05-06,NaN,100.0,5.3,NaN,NaN,1990,[English Language],[Comedy],[United States of America]


In [238]:
movies.shape

(19361, 13)

1.6 Genres

No need to reformat as it was done in the first steps of the Languages section.

Keep only certain genres above a selected threshold. Gets rid of 1067 movies.

In [239]:
# Counts the numbers of appearance of each genre in all movies
exploded_genres = movies['Genres'].explode()
genre_counts = exploded_genres.value_counts()

# A quick look at the numbers of appearance of each genre in all movies
genre_counts

Genres
Drama                   9539
Comedy                  5949
Romance Film            3778
Thriller                3742
Action                  3174
                        ... 
Race movie                 1
Chick flick                1
Political Documetary       1
Patriotic film             1
World History              1
Name: count, Length: 339, dtype: int64

In [240]:
# List of genres before threshold filtering
unique_genres_list = genre_counts.index.tolist()

unique_genres_list

['Drama',
 'Comedy',
 'Romance Film',
 'Thriller',
 'Action',
 'Crime Fiction',
 'World cinema',
 'Black-and-white',
 'Action/Adventure',
 'Indie',
 'Adventure',
 'Horror',
 'Family Film',
 'Romantic drama',
 'Science Fiction',
 'Romantic comedy',
 'Mystery',
 'Musical',
 'Fantasy',
 'Crime Thriller',
 'Animation',
 'Period piece',
 'Short Film',
 'Film adaptation',
 'Comedy-drama',
 'War film',
 'Psychological thriller',
 'Japanese Movies',
 'Bollywood',
 'Comedy film',
 'Teen',
 "Children's/Family",
 'Silent film',
 'Black comedy',
 'Western',
 'Coming of age',
 'Family Drama',
 'Sports',
 'Parody',
 'Cult',
 'Suspense',
 'LGBT',
 'Biographical film',
 'Slapstick',
 'Documentary',
 'Biography',
 'Film noir',
 'Television movie',
 'Chinese Movies',
 'Satire',
 'Supernatural',
 'Slasher',
 'Political drama',
 "Children's",
 'Melodrama',
 'Martial Arts Film',
 'Buddy film',
 'Costume drama',
 'Action Thrillers',
 'Biopic [feature]',
 'Ensemble Film',
 'Crime Drama',
 'History',
 'Gangst

In [241]:
# Threshold of 1'000
threshold = 1000 # à voir si on le change un peu
# Filter out genres below the threshold
filtered_genres = genre_counts[genre_counts >= threshold].index.tolist()

# Creates filter for the original DataFrame based on the threshold
filtered_genres = movies.apply(lambda row: [genre for genre in row['Genres'] if genre in filtered_genres], axis=1)

In [242]:
# Inject the modified "Genres"
movies["Genres"] = filtered_genres

<ipython-input-242-8523aeffb33b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["Genres"] = filtered_genres


In [243]:
# Counts the counts for each genre in all movies after threshold filter
exploded_genres = movies['Genres'].explode()
genre_counts = exploded_genres.value_counts()
genre_counts

Genres
Drama               9539
Comedy              5949
Romance Film        3778
Thriller            3742
Action              3174
Crime Fiction       2426
World cinema        2248
Black-and-white     2077
Action/Adventure    2016
Indie               1965
Adventure           1950
Horror              1828
Family Film         1698
Romantic drama      1471
Science Fiction     1328
Romantic comedy     1314
Mystery             1289
Musical             1276
Fantasy             1209
Crime Thriller      1012
Name: count, dtype: int64

Avoids double counting of genres and missing mentions of certain mentions too.

For example:

List = [`Comedy-drama`, `Comedy`, `Drama`] results in 1 mention for `Comedy` and 1 mention for `Drama`.

List = [`Drama`, `Comedy`] results in 1 mention for `Comedy` and 1 mention for `Drama`

List = [`Comedy-drama`] results in 1 mention for `Comedy` and 1 mentions for `Drama`

The same way of thinking is implented for various cases

In [244]:
movies.shape

(19361, 13)

In [245]:
for index, row in movies.iterrows():
    if 'Comedy-drama' in row['Genres']:
        if 'Comedy' in row['Genres'] and 'Drama' in row['Genres']:
            row['Genres'].remove('Comedy-drama')
        elif 'Comedy' in row['Genres'] and 'Drama' not in row['Genres']:
            row['Genres'][row['Genres'].index('Comedy-drama')] = 'Drama'
        elif 'Drama' in row['Genres'] and 'Comedy' not in row['Genres']:
            row['Genres'][row['Genres'].index('Comedy-drama')] = 'Comedy'
        else:
            row['Genres'].remove('Comedy-drama')
            row['Genres'].extend(['Comedy', 'Drama'])

In [246]:
for index, row in movies.iterrows():
    if 'Romantic comedy' in row['Genres']:
        if 'Romance Film' in row['Genres'] and 'Comedy' in row['Genres']:
            row['Genres'].remove('Romantic comedy')
        elif 'Comedy' in row['Genres'] and 'Romance Film' not in row['Genres']:
            row['Genres'][row['Genres'].index('Romantic comedy')] = 'Romance Film'
        elif 'Romance Film' in row['Genres'] and 'Comedy' not in row['Genres']:
            row['Genres'][row['Genres'].index('Romantic comedy')] = 'Comedy'
        else:
            row['Genres'].remove('Romantic comedy')
            row['Genres'].extend(['Comedy', 'Romance Film'])

In [247]:
for index, row in movies.iterrows():
    if 'Romantic drama' in row['Genres']:
        if 'Romance Film' in row['Genres'] and 'Drama' in row['Genres']:
            row['Genres'].remove('Romantic drama')
        elif 'Drama' in row['Genres'] and 'Romance Film' not in row['Genres']:
            row['Genres'][row['Genres'].index('Romantic drama')] = 'Romance Film'
        elif 'Romance Film' in row['Genres'] and 'Drama' not in row['Genres']:
            row['Genres'][row['Genres'].index('Romantic drama')] = 'Drama'
        else:
            row['Genres'].remove('Romantic drama')
            row['Genres'].extend(['Drama', 'Romance Film'])

In [248]:
for index, row in movies.iterrows():
    if 'Action/Adventure' in row['Genres']:
        if 'Action' in row['Genres'] and 'Adventure' in row['Genres']:
            row['Genres'].remove('Action/Adventure')
        elif 'Action' in row['Genres'] and 'Adventure' not in row['Genres']:
            row['Genres'][row['Genres'].index('Action/Adventure')] = 'Adventure'
        elif 'Adventure' in row['Genres'] and 'Action' not in row['Genres']:
            row['Genres'][row['Genres'].index('Action/Adventure')] = 'Action'
        else:
            row['Genres'].remove('Action/Adventure')
            row['Genres'].extend(['Action', 'Adventure'])

In [249]:
for index, row in movies.iterrows():
    if 'Comedy film' in row['Genres']:
        if 'Comedy' in row['Genres']:
            row['Genres'].remove('Comedy film')
        else:
            row['Genres'].remove('Comedy film')
            row['Genres'].extend('Comedy')

In [250]:
# Filter 'movies' DataFrame to retain rows where the 'Genres' column has non-empty values
movies = movies[movies['Genres'].astype(bool)]
# Display the first 30 rows of the filtered DataFrame
movies.head(30)

,Wiki ID,Freebase ID,Movie name,Release date,Revenue,Runtime,IMDb rating,Wins,Nominations,Release year,Languages,Genres,Countries
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,2.315268e+07,98.0,4.9,NaN,2.0,2001,[English Language],"[Thriller, Science Fiction, Horror, Adventure,...",[United States of America]
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,5.8,NaN,NaN,1997,[English Language],"[Drama, Comedy, Romance Film]",[United States of America]
2,171005,/m/016ywb,Henry V,1989-11-08,2.398142e+07,137.0,7.5,12.0,13.0,1989,[English Language],[Drama],[United Kingdom]
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,9.655040e+08,139.0,7.8,22.0,17.0,1964,[English Language],"[Musical, Fantasy, Comedy, Drama, Family Film]",[United States of America]
4,20604092,/m/0523t_1,Anbu Thozhi,2007-08-17,NaN,NaN,NaN,NaN,NaN,2007,[Tamil Language],[Romance Film],[India]
5,156558,/m/014k4y,Baby Boy,2001-06-27,4.855272e+07,123.0,6.4,1.0,14.0,2001,[English Language],"[Crime Fiction, Drama]",[United States of America]
6,11448183,/m/02rc_h4,Bindiya Chamkegi,1984-01-20,NaN,NaN,4.8,NaN,NaN,1984,[Hindi Language],"[Family Film, Comedy]",[India]
7,29772142,/m/0fq260_,Karayilekku Oru Kadal Dooram,2010-12-31,NaN,110.0,6.1,NaN,NaN,2010,[Malayalam Language],[Drama],[India]
8,26623942,/m/0bh9fbk,Kausthubham,2010-03-19,NaN,NaN,NaN,NaN,NaN,2010,[Malayalam Language],[Drama],[India]
9,9548445,/m/02pjlrp,Archie: To Riverdale and Back Again,1990-05-06,NaN,100.0,5.3,NaN,NaN,1990,[English Language],[Comedy],[United States of America]


1.7 Wins and nominations

We get rid of outliers. Gets rid of 6 movies.

In [251]:
movies.shape

(18294, 13)

In [252]:
# Apply outlier filtering to the 'movies' DataFrame, focusing on the 'Wins' column, retaining values between 0 and 200.
movies = Outliers_filtering(movies, 'Wins', 0, 200)

# Further filter the 'movies' DataFrame, specifically targeting the 'Nominations' column,
# keeping values within the range of 0 to 300 using the 'Outliers_filtering' function.
movies = Outliers_filtering(movies, 'Nominations', 0, 300)

In [253]:
movies.shape

(18288, 13)

1.8 Onehot encoding of categorical features

Languages

In [254]:
movies.shape

(18288, 13)

In [255]:

# Explode the 'movies' DataFrame based on the 'Languages' column, creating a new DataFrame 'language_explode'
language_explode = movies.explode('Languages')

# Extract unique language values from the 'Languages' column of the 'language_explode' DataFrame
# and store them in the 'language_unique' variable, representing unique languages.
language_unique = language_explode['Languages'].unique()  # unique languages

# Append '-onehot' to each unique language name and store them in 'language_track_names' for potential one-hot encoding.
language_track_names = language_unique + "-onehot"  # unique languages with '-onehot' appended

# Display the unique language values
language_unique
# Uncomment the line below if you want to display the variable 'language_track_names'
# language_track_names


array(['English Language', 'Tamil Language', 'Hindi Language',
       'Malayalam Language', 'German Language', 'Japanese Language',
       'Korean Language', 'Spanish Language', 'Italian Language',
       'French Language', 'Standard Mandarin', 'Standard Cantonese',
       'Bengali Language', 'Cantonese', 'Telugu language',
       'Russian Language', 'Portuguese Language', 'Urdu Language',
       'Arabic Language', 'Tagalog language', 'Swedish Language'],
      dtype=object)

In [256]:

# Ajoutez des colonnes avec les noms de track_names, initialisées à 0
for name in language_track_names:
    movies[name] = 0

# We transform track_names 
language_track_names_columns = language_track_names.tolist()

# Iterate over each row and set one-hot encoded columns
for index, row in movies.iterrows(): # iteration over indexes and rows of data_one_hot
    if row['Languages'] is not np.nan: # condition: if the value is not a Nan, we can enter
        #print(index) # we print to make sure that we enter the loop (à supprimer)
        languages = row['Languages'] # we extract lists of the genre for each row
        for language in languages: # we check fror the different genres in the list
            #print(genre) # we print the genre (à supprimer)
            if language +'-onehot' in language_track_names_columns: # condition if the genre-onehot is present in track_names_colums
                #print('je rentre') # we make sure we enter the loop (à supprimer)
                movies.at[index, language+'-onehot'] = 1 

# We display the dataframe


Countries

In [257]:
movies.shape

(18288, 34)

In [258]:
#exploser notre dataframe sur les countries 
country_explode = movies.explode('Countries')

#Récupérer les valeurs uniques des countries 
country_unique = country_explode['Countries'].unique() #les countries uniques 

#Récupérer les valeurs uniques des countries-onhot 
country_track_names = country_unique+"-onehot" #les countries uniques + onehot à la fin

# Ajoutez des colonnes avec les noms de country_track_names, initialisées à 0
for name in country_track_names:
    movies[name] = 0


country_track_names_columns = country_track_names.tolist()

# Iterate over each row and set one-hot encoded columns
for index, row in movies.iterrows(): #parcouri les index de data_final et les lignes
    if row['Countries'] is not np.nan: #si la valeur de genre n'est pas un nan on rentre
        #print(index) #on print l'index de la ligne en question
        
        countries = row['Countries'] #extrait les genre de la ligne 1 sous forme de liste 
    
        #print(genres) #imprime cette liste

        for country in countries: #on parcourt la liste de genre issu du film 
            #print(country) #Imprimer le genre en question
            if country +'-onehot' in country_track_names_columns: # Si ce genre +onehot est bien dans la list des track_names_columns
                #print('je rentre') #on est sûrs qu'on rentre 
                movies.at[index, country+'-onehot'] = 1 #si on est bien rentrés on attribue la valeur 1 


movies #on display 

,Wiki ID,Freebase ID,Movie name,Release date,Revenue,Runtime,IMDb rating,Wins,Nominations,Release year,...,Australia-onehot,Italy-onehot,Mexico-onehot,New Zealand-onehot,Brazil-onehot,Belgium-onehot,Denmark-onehot,Sweden-onehot,Netherlands-onehot,Philippines-onehot
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,2.315268e+07,98.0,4.9,NaN,2.0,2001,...,0,0,0,0,0,0,0,0,0,0
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,5.8,NaN,NaN,1997,...,0,0,0,0,0,0,0,0,0,0
2,171005,/m/016ywb,Henry V,1989-11-08,2.398142e+07,137.0,7.5,12.0,13.0,1989,...,0,0,0,0,0,0,0,0,0,0
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,9.655040e+08,139.0,7.8,22.0,17.0,1964,...,0,0,0,0,0,0,0,0,0,0
4,20604092,/m/0523t_1,Anbu Thozhi,2007-08-17,NaN,NaN,NaN,NaN,NaN,2007,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22360,15394941,/m/03m6zh4,Gopi Kishan,1994-12-02,NaN,NaN,5.8,NaN,NaN,1994,...,0,0,0,0,0,0,0,0,0,0
22362,23851782,/m/06_vb43,The Ghost Train,1941-05-03,NaN,82.0,6.2,NaN,NaN,1941,...,0,0,0,0,0,0,0,0,0,0
22363,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,4.6,NaN,NaN,2011,...,0,0,0,0,0,0,0,0,0,0
22364,34980460,/m/0g4pl34,Knuckle,2011-01-21,3.443857e+03,96.0,6.8,1.0,4.0,2011,...,0,0,0,0,0,0,0,0,0,0


Genres

In [259]:
movies.shape

(18288, 59)

In [260]:
# We explode the datafrme to obtain every genre separated, only linked thanks to the Wiki ID
genre_explode = movies.explode('Genres')
# We first select the names of the different "tracks-onehot" columns 
genre_unique = genre_explode['Genres'].unique()
track_names = genre_explode['Genres'].unique()+"-onehot"

genre_unique

array(['Thriller', 'Science Fiction', 'Horror', 'Adventure', 'Action',
       'Drama', 'Comedy', 'Romance Film', 'Musical', 'Fantasy',
       'Family Film', 'Crime Fiction', 'Indie', 'World cinema', 'Mystery',
       'Black-and-white', 'Crime Thriller'], dtype=object)

In [261]:
# We add columns containing track_names, initialized with 0s
for name in track_names:
    movies[name] = 0

# We transform track_names 
track_names_columns = track_names.tolist()

# Iterate over each row and set one-hot encoded columns
for index, row in movies.iterrows(): # iteration over indexes and rows of data_one_hot
    if row['Genres'] is not np.nan: # condition: if the value is not a Nan, we can enter
        #print(index) # we print to make sure that we enter the loop (à supprimer)
        genres = row['Genres'] # we extract lists of the genre for each row
        for genre in genres: # we check fror the different genres in the list
            #print(genre) # we print the genre (à supprimer)
            if genre +'-onehot' in track_names_columns: # condition if the genre-onehot is present in track_names_colums
                #print('je rentre') # we make sure we enter the loop (à supprimer)
                movies.at[index, genre+'-onehot'] = 1 

# We display the dataframe
movies

,Wiki ID,Freebase ID,Movie name,Release date,Revenue,Runtime,IMDb rating,Wins,Nominations,Release year,...,Romance Film-onehot,Musical-onehot,Fantasy-onehot,Family Film-onehot,Crime Fiction-onehot,Indie-onehot,World cinema-onehot,Mystery-onehot,Black-and-white-onehot,Crime Thriller-onehot
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,2.315268e+07,98.0,4.9,NaN,2.0,2001,...,0,0,0,0,0,0,0,0,0,0
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,5.8,NaN,NaN,1997,...,1,0,0,0,0,0,0,0,0,0
2,171005,/m/016ywb,Henry V,1989-11-08,2.398142e+07,137.0,7.5,12.0,13.0,1989,...,0,0,0,0,0,0,0,0,0,0
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,9.655040e+08,139.0,7.8,22.0,17.0,1964,...,0,1,1,1,0,0,0,0,0,0
4,20604092,/m/0523t_1,Anbu Thozhi,2007-08-17,NaN,NaN,NaN,NaN,NaN,2007,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22360,15394941,/m/03m6zh4,Gopi Kishan,1994-12-02,NaN,NaN,5.8,NaN,NaN,1994,...,0,0,0,0,0,0,0,0,0,0
22362,23851782,/m/06_vb43,The Ghost Train,1941-05-03,NaN,82.0,6.2,NaN,NaN,1941,...,0,0,0,0,1,0,0,0,0,0
22363,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,4.6,NaN,NaN,2011,...,0,0,0,0,0,0,0,0,0,0
22364,34980460,/m/0g4pl34,Knuckle,2011-01-21,3.443857e+03,96.0,6.8,1.0,4.0,2011,...,0,0,0,0,0,0,0,0,0,0


2. Clean `characters`

2.1 Release date

Keep Release dates after 1913 as done for `movies`. Gets rid of 26 movies.

In [262]:
characters.shape

(34000, 14)

In [263]:
def custom_date_parser(date_str):
    try:
        # Attempt to parse as a full date
        return pd.to_datetime(date_str, errors='coerce')
    except ValueError:
        # If parsing as a full date fails, parse as a year-only date
        return pd.to_datetime(date_str + '-01-01', errors='coerce')

# Apply the custom_date_parser function to the 'Release date' column
characters['Release date'] = characters['Release date'].apply(custom_date_parser)

In [264]:
# Filter the 'characters' DataFrame to retain rows where the 'Release date' year is greater than or equal to 1913,
# or where the 'Release date' year is missing (null/NaN).
characters = characters[(characters['Release date'].dt.year >= 1913) | (characters['Release date'].dt.year.isnull())]

In [265]:
characters.shape

(33974, 14)

2.2 Actor height

Keep Actor height within acceptable interval. Gets rid of 2.

In [266]:
# Use the 'Outliers_filtering' function to filter outliers in the 'characters' DataFrame,
# specifically focusing on the 'Actor height' column, retaining values within the range of 0.3 to 2.5.
characters = Outliers_filtering(characters, 'Actor height', 0.3, 2.5)

In [267]:
characters.shape

(33972, 14)

2.3 Actor age at release

Fill by “Release date” – “Actor DOB”

In [268]:
def filter_actor_dob(dataframe):
    # Convert 'Actor DOB' column to datetime with coerce to handle invalid dates
    dataframe['Actor DOB'] = pd.to_datetime(dataframe['Actor DOB'], errors='coerce', utc=True)

    # Extract year from valid dates and handle irregular formats
    dataframe['Year'] = dataframe['Actor DOB'].dt.strftime('%Y')  # Extract year from valid dates

    # Convert years as integers for further manipulation
    dataframe['Year'] = pd.to_numeric(dataframe['Year'], errors='coerce')

    # Filter rows where the year is greater than 1913 or it's NaN
    return dataframe

characters = filter_actor_dob(characters)


In [269]:
characters.shape

(33972, 15)

In [270]:
def extract_year(date):
    if pd.isnull(date):  # Check if NaN, return NaN
        return np.nan
    elif isinstance(date, str) and len(date) > 4:  # Extract year from date string
        return pd.to_datetime(date).year
    elif isinstance(date, (pd.Timestamp, (pd.Timestamp, pd._libs.tslibs.timestamps.Timestamp))):  # Extract year from datetime object
        return date.year
    else:  # Return year as is
        return date

# Apply the function to 'Release date' and 'Actor DOB' columns
characters['Release year'] = characters['Release date'].apply(extract_year)  # Extract release year
print(characters.shape)
characters['Actor birth year'] = characters['Actor DOB'].apply(extract_year)  # Extract actor birth year
print(characters.shape)

# Calculate the age at release using the extracted years
characters['Age at Release'] = characters['Release year'] - characters['Actor birth year']  # Calculate age at release

# Fill 'Actor age at release' with 'Age at Release' if NaN, else keep existing value
characters['Actor age at release'] = characters.apply(
    lambda row: row['Age at Release'] if pd.isnull(row['Actor age at release']) else row['Actor age at release'],
    axis=1
)

# Drop 'Age at Release' column (optional)
characters.drop('Age at Release', axis=1, inplace=True)  # Drop intermediate 'Age at Release' column

characters['Actor DOB'] = characters['Actor DOB'].dt.date  # Convert 'Actor DOB' to date format

# Display the updated DataFrame
characters

(33972, 16)
(33972, 17)


,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,Map ID,Character ID,Actor ID,Role,Year,Release year,Actor birth year
0,5894429,/m/0fc8w8,2007-09-14,Ethel Ann,1934-04-24,F,1.700,NaN,Shirley MacLaine,73.0,/m/0jwjk9,/m/0h129m6,/m/01w1kyf,Primary,1934.0,2007.0,1934.0
1,5894429,/m/0fc8w8,2007-09-14,Young Ethel Ann,1986-01-24,F,1.750,/m/09vc4s,Mischa Barton,21.0,/m/0jwjkg,/m/0gy6lpz,/m/01qn8k,Secondary,1986.0,2007.0,1986.0
2,27556929,/m/04j0jtp,1996-01-01,Kingsley Ofusu,1973-07-20,M,1.778,/m/0x67,Omar Epps,22.0,/m/04j0jts,/m/0h2j4r7,/m/02_tv5,Secondary,1973.0,1996.0,1973.0
3,27556929,/m/04j0jtp,1996-01-01,Vlachos,1946-05-02,M,1.700,/m/02w7gg,David Suchet,49.0,/m/0cg0qq4,/m/0h2t3nh,/m/013bd1,Primary,1946.0,1996.0,1946.0
4,27463222,/m/0c037x9,2010-09-12,Paul,1964-07-22,M,1.730,/m/025rpb0,John Leguizamo,46.0,/m/0c05h8f,/m/0gdjxq3,/m/04yj5z,Secondary,1964.0,2010.0,1964.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33995,18577105,/m/04f_3ym,1990-01-01,Katya Orlova,1958-04-29,F,1.710,/m/065b6q,Michelle Pfeiffer,31.0,/m/03lbwrf,/m/0gykcf1,/m/0gx_p,Primary,1958.0,1990.0,1958.0
33996,454812,/m/02bjfb,1971-08-03,Quentin Collins,1941-02-05,M,1.900,NaN,David Selby,30.0,/m/02vddsx,/m/0h_6h1b,/m/04pvw3,Primary,1941.0,1971.0,1941.0
33997,454812,/m/02bjfb,1971-08-03,Tracy Collins,1948-10-29,F,1.750,NaN,Kate Jackson,22.0,/m/02vbj0g,/m/0h3c6mg,/m/01phkr,Secondary,1948.0,1971.0,1948.0
33998,25920477,/m/0b6lqyd,2011-03-11,Colter Stevens,1980-12-19,M,1.830,/m/065b6q,Jake Gyllenhaal,30.0,/m/0cpl4h_,/m/0ggkqv7,/m/02js6_,Primary,1980.0,2011.0,1980.0


In [271]:
characters.shape

(33972, 17)

Keep Actor age at release within an acceptable range. Gets rid of 9.

In [272]:
# Apply outlier filtering to the 'characters' DataFrame based on the 'Actor age at release' column,
# keeping values within the range of 0 to 100 using the 'Outliers_filtering' function.
characters = Outliers_filtering(characters, 'Actor age at release', 0, 100)

In [273]:
characters.shape

(33963, 17)

2.4 Etnicities 

We load the ethnicities based on Freebase ID

In [274]:
ethnicities = pd.read_csv("../Clean data/map_ethnicity.csv")

In [275]:
dict_from_df = ethnicities.to_dict(orient='records') 
dict_from_df

[{'Actor ethnicity': nan, 'Ethnicity': 'Unknown'},
 {'Actor ethnicity': '/m/09vc4s', 'Ethnicity': 'English Americans'},
 {'Actor ethnicity': '/m/0x67', 'Ethnicity': 'African Americans'},
 {'Actor ethnicity': '/m/02w7gg', 'Ethnicity': 'English people'},
 {'Actor ethnicity': '/m/025rpb0',
  'Ethnicity': 'Hispanic and Latino Americans'},
 {'Actor ethnicity': '/m/092h2qt', 'Ethnicity': 'Unknown'},
 {'Actor ethnicity': '/m/07bch9', 'Ethnicity': 'Scottish American'},
 {'Actor ethnicity': '/m/0dryh9k', 'Ethnicity': 'Indian person'},
 {'Actor ethnicity': '/m/0f0gt_', 'Ethnicity': 'Asian people'},
 {'Actor ethnicity': '/m/06gbnc', 'Ethnicity': 'Welsh people'},
 {'Actor ethnicity': '/m/07hwkr', 'Ethnicity': 'White Americans'},
 {'Actor ethnicity': '/m/09kr66', 'Ethnicity': 'Russian Americans'},
 {'Actor ethnicity': '/m/07mqps', 'Ethnicity': 'Dutch Americans'},
 {'Actor ethnicity': '/m/0d7wh', 'Ethnicity': 'British people'},
 {'Actor ethnicity': '/m/041rx', 'Ethnicity': 'Jewish people'},
 {'Actor

In [276]:
result_dict = {}
for item in dict_from_df:
    if 'Actor ethnicity' in item:
        result_dict[item['Actor ethnicity']] = item['Ethnicity']

print(result_dict)

{nan: 'Unknown', '/m/09vc4s': 'English Americans', '/m/0x67': 'African Americans', '/m/02w7gg': 'English people', '/m/025rpb0': 'Hispanic and Latino Americans', '/m/092h2qt': 'Unknown', '/m/07bch9': 'Scottish American', '/m/0dryh9k': 'Indian person', '/m/0f0gt_': 'Asian people', '/m/06gbnc': 'Welsh people', '/m/07hwkr': 'White Americans', '/m/09kr66': 'Russian Americans', '/m/07mqps': 'Dutch Americans', '/m/0d7wh': 'British people', '/m/041rx': 'Jewish people', '/m/0xnvg': 'Italian Americans', '/m/03bx0k4': 'Hongkongers', '/m/0fj1k9': 'Sicilian Americans', '/m/01qhm_': 'German Americans', '/m/08hpk0': 'Hungarian Americans', '/m/0dqqwy': 'Danish Americans', '/m/0g8_vp': 'Canadian Americans', '/m/044038p': 'Unknown', '/m/02sch9': 'Punjabis', '/m/02ctzb': 'white people', '/m/033tf_': 'Irish Americans', '/m/05748': 'Māori', '/m/03ts0c': 'French', '/m/065b6q': 'Swedish Americans', '/m/0cnvdq1': 'Scandinavian Americans', '/m/02q206y': 'Iranians in the United Kingdom', '/m/06v41q': 'French Am

In [277]:
characters.shape

(33963, 17)

In [278]:
characters['Actor ethnicity'] = characters['Actor ethnicity'].map(result_dict)
characters

<ipython-input-278-82dba530e545>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  characters['Actor ethnicity'] = characters['Actor ethnicity'].map(result_dict)


,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,Map ID,Character ID,Actor ID,Role,Year,Release year,Actor birth year
0,5894429,/m/0fc8w8,2007-09-14,Ethel Ann,1934-04-24,F,1.700,Unknown,Shirley MacLaine,73.0,/m/0jwjk9,/m/0h129m6,/m/01w1kyf,Primary,1934.0,2007.0,1934.0
1,5894429,/m/0fc8w8,2007-09-14,Young Ethel Ann,1986-01-24,F,1.750,English Americans,Mischa Barton,21.0,/m/0jwjkg,/m/0gy6lpz,/m/01qn8k,Secondary,1986.0,2007.0,1986.0
2,27556929,/m/04j0jtp,1996-01-01,Kingsley Ofusu,1973-07-20,M,1.778,African Americans,Omar Epps,22.0,/m/04j0jts,/m/0h2j4r7,/m/02_tv5,Secondary,1973.0,1996.0,1973.0
3,27556929,/m/04j0jtp,1996-01-01,Vlachos,1946-05-02,M,1.700,English people,David Suchet,49.0,/m/0cg0qq4,/m/0h2t3nh,/m/013bd1,Primary,1946.0,1996.0,1946.0
4,27463222,/m/0c037x9,2010-09-12,Paul,1964-07-22,M,1.730,Hispanic and Latino Americans,John Leguizamo,46.0,/m/0c05h8f,/m/0gdjxq3,/m/04yj5z,Secondary,1964.0,2010.0,1964.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33995,18577105,/m/04f_3ym,1990-01-01,Katya Orlova,1958-04-29,F,1.710,Swedish Americans,Michelle Pfeiffer,31.0,/m/03lbwrf,/m/0gykcf1,/m/0gx_p,Primary,1958.0,1990.0,1958.0
33996,454812,/m/02bjfb,1971-08-03,Quentin Collins,1941-02-05,M,1.900,Unknown,David Selby,30.0,/m/02vddsx,/m/0h_6h1b,/m/04pvw3,Primary,1941.0,1971.0,1941.0
33997,454812,/m/02bjfb,1971-08-03,Tracy Collins,1948-10-29,F,1.750,Unknown,Kate Jackson,22.0,/m/02vbj0g,/m/0h3c6mg,/m/01phkr,Secondary,1948.0,1971.0,1948.0
33998,25920477,/m/0b6lqyd,2011-03-11,Colter Stevens,1980-12-19,M,1.830,Swedish Americans,Jake Gyllenhaal,30.0,/m/0cpl4h_,/m/0ggkqv7,/m/02js6_,Primary,1980.0,2011.0,1980.0


2.5 Onehot encoding of categorical features

Actor gender

In [279]:
# One-hot encoding of the 'Actor gender' column
one_hot_encoded_track = pd.get_dummies(characters['Actor gender'])

# Convert boolean 'one_hot_encoded_track' columns to integers (0s and 1s)
one_hot_encoded_track = one_hot_encoded_track.astype(int)

# Renaming the new columns to {track}-onehot
one_hot_encoded_track.columns = [f"{col}-{'onehot'}" for col in one_hot_encoded_track.columns]

# Replace one-hot encoded columns where original gender column is NaN with NaN
nan_mask = characters['Actor gender'].isna()
one_hot_encoded_track[nan_mask] = float('nan')

# Concatenate the one-hot encoded columns with the original DataFrame
characters = pd.concat([characters, one_hot_encoded_track], axis=1)

# Display the updated DataFrame
characters


,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,Map ID,Character ID,Actor ID,Role,Year,Release year,Actor birth year,F-onehot,M-onehot
0,5894429,/m/0fc8w8,2007-09-14,Ethel Ann,1934-04-24,F,1.700,Unknown,Shirley MacLaine,73.0,/m/0jwjk9,/m/0h129m6,/m/01w1kyf,Primary,1934.0,2007.0,1934.0,1.0,0.0
1,5894429,/m/0fc8w8,2007-09-14,Young Ethel Ann,1986-01-24,F,1.750,English Americans,Mischa Barton,21.0,/m/0jwjkg,/m/0gy6lpz,/m/01qn8k,Secondary,1986.0,2007.0,1986.0,1.0,0.0
2,27556929,/m/04j0jtp,1996-01-01,Kingsley Ofusu,1973-07-20,M,1.778,African Americans,Omar Epps,22.0,/m/04j0jts,/m/0h2j4r7,/m/02_tv5,Secondary,1973.0,1996.0,1973.0,0.0,1.0
3,27556929,/m/04j0jtp,1996-01-01,Vlachos,1946-05-02,M,1.700,English people,David Suchet,49.0,/m/0cg0qq4,/m/0h2t3nh,/m/013bd1,Primary,1946.0,1996.0,1946.0,0.0,1.0
4,27463222,/m/0c037x9,2010-09-12,Paul,1964-07-22,M,1.730,Hispanic and Latino Americans,John Leguizamo,46.0,/m/0c05h8f,/m/0gdjxq3,/m/04yj5z,Secondary,1964.0,2010.0,1964.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33995,18577105,/m/04f_3ym,1990-01-01,Katya Orlova,1958-04-29,F,1.710,Swedish Americans,Michelle Pfeiffer,31.0,/m/03lbwrf,/m/0gykcf1,/m/0gx_p,Primary,1958.0,1990.0,1958.0,1.0,0.0
33996,454812,/m/02bjfb,1971-08-03,Quentin Collins,1941-02-05,M,1.900,Unknown,David Selby,30.0,/m/02vddsx,/m/0h_6h1b,/m/04pvw3,Primary,1941.0,1971.0,1941.0,0.0,1.0
33997,454812,/m/02bjfb,1971-08-03,Tracy Collins,1948-10-29,F,1.750,Unknown,Kate Jackson,22.0,/m/02vbj0g,/m/0h3c6mg,/m/01phkr,Secondary,1948.0,1971.0,1948.0,1.0,0.0
33998,25920477,/m/0b6lqyd,2011-03-11,Colter Stevens,1980-12-19,M,1.830,Swedish Americans,Jake Gyllenhaal,30.0,/m/0cpl4h_,/m/0ggkqv7,/m/02js6_,Primary,1980.0,2011.0,1980.0,0.0,1.0


Actor ethnicity (uses Chat GPT for clustering and requires manual verification)

In [280]:
# Extract unique values present in the 'Actor ethnicity' column of the 'characters' DataFrame
unique_ethnicities = characters['Actor ethnicity'].unique()

# Convert unique values to a list and print
print(list(unique_ethnicities))


['Unknown', 'English Americans', 'African Americans', 'English people', 'Hispanic and Latino Americans', 'Scottish American', 'Indian person', 'Asian people', 'Welsh people', 'White Americans', 'Russian Americans', 'Dutch Americans', 'British people', 'Jewish people', 'Italian Americans', 'Hongkongers', 'Sicilian Americans', 'German Americans', 'Hungarian Americans', 'Danish Americans', 'Canadian Americans', 'Punjabis', 'white people', 'Irish Americans', 'Māori', 'French', 'Swedish Americans', 'Scandinavian Americans', 'Iranians in the United Kingdom', 'French Americans', 'Germans', 'American Jews', 'Scottish people', 'names of the Greeks', 'Swedes', 'Malayali', 'Australian American', 'Armenians', 'Marathi people', 'Tamil', 'Australians', 'Greek American', 'Stateside Puerto Ricans', 'Serbian Americans', 'Indian Americans', 'Black people', 'Mexican Americans', 'Koreans', 'Anglo-Celtic Australians', 'Akan people', 'Irish people', 'Nair', 'Portuguese Americans', 'Bengali people', 'Cajun',

In [281]:
cluster_mapping = {
    'White': ['English Americans', 'English people', 'Scottish American', 'Welsh people', 'White Americans', 'Russian Americans', 'Dutch Americans', 'British people', 'Jewish people', 'Italian Americans', 'Sicilian Americans', 'German Americans', 'Hungarian Americans', 'Danish Americans', 'Canadian Americans', 'French', 'Swedish Americans', 'Scandinavian Americans', 'Iranians in the United Kingdom', 'French Americans', 'Germans', 'American Jews', 'Scottish people', 'names of the Greeks', 'Swedes', 'Australian American', 'Australians', 'Greek American', 'Serbian Americans', 'Anglo-Celtic Australians', 'Irish people', 'Portuguese Americans', 'Cajun', 'White British', 'Italians', 'Italian Australian', 'Scotch-Irish Americans', 'Danes', 'Slovak Americans', 'Anglo-Irish people', 'European Americans', 'Irish migration to Great Britain', 'Welsh American', 'Bulgarian Canadians', 'Cherokee', 'Eurasian', 'Serbs of Croatia', 'Americans', 'Serbian Canadians', 'Romanian Americans', 'Swedish Canadians', 'Slovene American', 'Italians in the United Kingdom', 'Italian Canadians', 'British Jews', 'Polish Canadians', 'Russian Canadians', 'Greek Canadians', 'Belarusians', 'Palestinian American', 'Irish Australian', 'Lithuanian American', 'Polish Australians', 'Estonians', 'White Africans of European ancestry', 'Russians', 'Dutch Canadians', 'Serbs of Bosnia and Herzegovina', 'Scottish Canadians', 'Austrians', 'Canadians in the United Kingdom', 'Czechs', 'Finnish Americans', 'Portuguese', 'Albanian American', 'Ukrainian Americans', 'Canadian Australian', 'Ukrainians', 'Norwegians', 'English Australian', 'Croatian Australians', 'Finns', 'Ashkenazi Jews', 'Israeli Americans', 'Mizrahi Jews', 'Norwegian Americans', 'Croats', 'Latvians', 'French Canadians', 'Czech Americans', 'Sephardi Jews', 'Dutch Australian', 'Croatian Americans', 'Swiss', 'Icelanders', 'Polish Americans', 'Dutch', 'Irish Canadians', 'Austrian Americans', 'British Americans', 'Israelis', 'Hungarians', 'Copts', 'Poles', 'Romanichal', 'Sámi peoples', 'History of the Jews in Morocco', 'Rusyn American', 'Lumbee', 'white people', 'Irish Americans', 'Spanish American', 'Spaniards', 'Italian Brazilians', 'French Chilean', 'Italian immigration to Mexico'],
    'Black/African American': ['African Americans', 'Black people', 'Akan people', 'Nigerian American', 'African people', 'Black Canadians', 'Black British', 'Black Irish', 'Somalis', 'Louisiana Creole people', 'Yoruba people', 'Xhosa people', 'British Nigerian', 'South African American', 'Ghanaian American'],
    'South Asian': ['Indian person', 'Punjabis', 'Malayali', 'Marathi people', 'Tamil', 'Nair', 'Bengali people', 'Bunt', 'Sikh', 'Gujarati people', 'Telugu people', 'Chaliyan', 'Nepali Indian', 'Filipino people', 'Sri Lankan Tamils', 'Bihari people', 'Vietnamese people', 'Sindhis', 'Kayastha', 'history of the Jews in India', 'Bengali Hindus', 'Ezhava', 'Kanyakubja Brahmins', 'Telugu Brahmins', 'Bhutia', 'Brahmin caste', 'Kashmiri Pandit', 'Pakistanis', 'Tamil Brahmin', 'Karnataka Brahmins', 'Chitrapur Saraswat Brahmin', 'Indian diaspora in France', 'Konkani people', 'Kashmiri people', 'Gaud Saraswat Brahmin', 'rajput', 'Ryukyuan people', 'Marwari people', 'Hindkowans', 'Javanese', 'Romani people', 'Jaat', 'Mangaloreans', 'Pathani', 'Mudaliar', 'Kannada people', 'Jatt Sikh', 'Niyogi', 'Sinhalese', 'Mohyal', 'Tulu people', 'Agrawal', 'Lao people', 'Hindu', 'Asian Americans', 'Filipino Americans', 'Indonesian Americans', 'Hmong American', 'Pakistani Americans', 'Indian Australian', 'Filipino Australians', 'Tamil Americans', 'Indo-Canadians'],
    'East Asian': ['Asian people', 'Hongkongers', 'Koreans', 'Chinese Singaporeans', 'Thai Chinese people', 'Chinese Filipino', 'Tibetan people', 'Rohilla', 'British Indian', 'Japanese people', 'Malaysian Chinese', 'Taiwanese', 'Manchu', 'Chinese Americans', 'Japanese Americans', 'Korean American', 'British Chinese', 'Taiwanese Americans', 'Chinese Canadians'], 
    'American Indian/Alaska Native': ['Indian Americans', 'Indigenous peoples of the Americas', 'Choctaw', 'Ojibwe', 'Inupiat people', 'Inuit', 'Dene', 'Native Americans in the United States', 'Bahamian Americans'],
    'Central and South Americans': ['Hispanic and Latino Americans', 'Stateside Puerto Ricans', 'Mexican Americans' , 'Colombians', 'Venezuelans', 'Afro Trinidadians and Tobagonians', 'Argentines', 'Brazilians', 'Chileans', 'Puerto Ricans', 'Cubans', 'Hispanic', 'Mexicans', 'Criollo people', 'Galicians', 'Latin American British', 'Colombian Americans', 'Cuban American', 'Bolivian American', 'Dominican Americans', 'Colombian Australian', 'Honduran Americans', 'Ecuadorian Americans', 'Salvadoran Americans', 'Guyanese American', 'Chilean American', 'Peruvians in the United Kingdom'], 
    'Pacific Islander': ['Māori', 'Kiwi', 'Indigenous Australians', 'Pacific Islander Americans', 'Native Hawaiians', 'Samoan American'], 
    'Arab/Middle East': ['Egyptians', 'Lebanese', 'Sudanese Arabs', 'Muslim', 'Kabyle people', 'Berber' , 'Armenians', 'Parsi', 'Iranians', 'Pashtuns', 'Ossetians', 'Hazaras', 'Afghans in India', 'Arab Americans', 'Turkish Americans',  'Syrian Americans', 'Lebanese Americans', 'Armenian American', 'Iraqi Americans', 'Moroccan Americans', 'Moroccans'],
    'Other': ['Unknown', 'Multiracial American', 'Afro-Asians']
}

In [282]:
# Reverse the mapping to create a dictionary where values map to keys
reverse_mapping = {value: key for key, values in cluster_mapping.items() for value in values}

# Use replace to map the elements in the column to their corresponding keys
characters['Actor ethnicity'] = characters['Actor ethnicity'].replace(reverse_mapping)

In [283]:
characters.shape

(33963, 19)

In [284]:
# Creating new columns for each ethnicity
ethnicities = ['White', 'Black/African American', 'South Asian', 'East Asian', 
               'American Indian/Alaska Native', 'Arab/Middle East', 'Central and South Americans', 'Pacific Islander', 'Other']

for ethnicity in ethnicities:
    characters[ethnicity] = characters['Actor ethnicity'].str.contains(ethnicity).astype(int)

In [285]:
characters.shape

(33963, 28)

7. Inputs formation

In [286]:
characters.columns

Index(['Wiki ID', 'Freebase ID', 'Release date', 'Character name', 'Actor DOB',
       'Actor gender', 'Actor height', 'Actor ethnicity', 'Actor name',
       'Actor age at release', 'Map ID', 'Character ID', 'Actor ID', 'Role',
       'Year', 'Release year', 'Actor birth year', 'F-onehot', 'M-onehot',
       'White', 'Black/African American', 'South Asian', 'East Asian',
       'American Indian/Alaska Native', 'Arab/Middle East',
       'Central and South Americans', 'Pacific Islander', 'Other'],
      dtype='object')

In [287]:
movies.columns

Index(['Wiki ID', 'Freebase ID', 'Movie name', 'Release date', 'Revenue',
       'Runtime', 'IMDb rating', 'Wins', 'Nominations', 'Release year',
       'Languages', 'Genres', 'Countries', 'English Language-onehot',
       'Tamil Language-onehot', 'Hindi Language-onehot',
       'Malayalam Language-onehot', 'German Language-onehot',
       'Japanese Language-onehot', 'Korean Language-onehot',
       'Spanish Language-onehot', 'Italian Language-onehot',
       'French Language-onehot', 'Standard Mandarin-onehot',
       'Standard Cantonese-onehot', 'Bengali Language-onehot',
       'Cantonese-onehot', 'Telugu language-onehot', 'Russian Language-onehot',
       'Portuguese Language-onehot', 'Urdu Language-onehot',
       'Arabic Language-onehot', 'Tagalog language-onehot',
       'Swedish Language-onehot', 'United States of America-onehot',
       'United Kingdom-onehot', 'India-onehot', 'Canada-onehot',
       'Germany-onehot', 'Japan-onehot', 'South Korea-onehot',
       'Argentina-one

In [288]:
movies.head()

,Wiki ID,Freebase ID,Movie name,Release date,Revenue,Runtime,IMDb rating,Wins,Nominations,Release year,...,Romance Film-onehot,Musical-onehot,Fantasy-onehot,Family Film-onehot,Crime Fiction-onehot,Indie-onehot,World cinema-onehot,Mystery-onehot,Black-and-white-onehot,Crime Thriller-onehot
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,2.315268e+07,98.0,4.9,NaN,2.0,2001,...,0,0,0,0,0,0,0,0,0,0
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,5.8,NaN,NaN,1997,...,1,0,0,0,0,0,0,0,0,0
2,171005,/m/016ywb,Henry V,1989-11-08,2.398142e+07,137.0,7.5,12.0,13.0,1989,...,0,0,0,0,0,0,0,0,0,0
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,9.655040e+08,139.0,7.8,22.0,17.0,1964,...,0,1,1,1,0,0,0,0,0,0
4,20604092,/m/0523t_1,Anbu Thozhi,2007-08-17,NaN,NaN,NaN,NaN,NaN,2007,...,1,0,0,0,0,0,0,0,0,0


In [289]:
characters.head()

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,...,M-onehot,White,Black/African American,South Asian,East Asian,American Indian/Alaska Native,Arab/Middle East,Central and South Americans,Pacific Islander,Other
0,5894429,/m/0fc8w8,2007-09-14,Ethel Ann,1934-04-24,F,1.700,Other,Shirley MacLaine,73.0,...,0.0,0,0,0,0,0,0,0,0,1
1,5894429,/m/0fc8w8,2007-09-14,Young Ethel Ann,1986-01-24,F,1.750,White,Mischa Barton,21.0,...,0.0,1,0,0,0,0,0,0,0,0
2,27556929,/m/04j0jtp,1996-01-01,Kingsley Ofusu,1973-07-20,M,1.778,Black/African American,Omar Epps,22.0,...,1.0,0,1,0,0,0,0,0,0,0
3,27556929,/m/04j0jtp,1996-01-01,Vlachos,1946-05-02,M,1.700,White,David Suchet,49.0,...,1.0,1,0,0,0,0,0,0,0,0
4,27463222,/m/0c037x9,2010-09-12,Paul,1964-07-22,M,1.730,Central and South Americans,John Leguizamo,46.0,...,1.0,0,0,0,0,0,0,1,0,0


In [290]:
# Splitting the 'characters' DataFrame based on the content of the 'Role' column:
# - 'primary_characters' contains rows where 'Role' has the term 'Primary'
# - 'secondary_characters' contains rows where 'Role' has the term 'Secondary'
primary_characters = characters[characters['Role'].str.contains('Primary')]
secondary_characters = characters[characters['Role'].str.contains('Secondary')]

In [291]:
# Merging 'movies' DataFrame with 'primary_characters' DataFrame based on 'Wiki ID' using an outer join
inputs = pd.merge(movies, primary_characters, on='Wiki ID', how='inner')

# Further merging 'inputs' DataFrame with 'secondary_characters' DataFrame based on 'Wiki ID' using an outer join
inputs = pd.merge(inputs, secondary_characters, on='Wiki ID', how='inner')


In [292]:
inputs.shape

(10918, 130)

In [293]:
characters.shape

(33963, 28)

In [294]:
primary_characters.shape

(16983, 28)

In [295]:
movies.shape

(18288, 76)

Export the data

In [296]:
characters.to_csv("../Clean data/clean_character_data.csv")
movies.to_csv("../Clean data/clean_movie_data.csv")
inputs.to_csv("../Clean data/inputs.csv", index=False)